## Data Characteristics

The data is an event log that was extracted from the audit system of a __ServiceNow__ platform (this is an enterprise service help desk application). The data is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Incident+management+process+enriched+event+log) (please visit the link for more details). This notebook captures the salient aspects of exploratory analysis of this dataset.

## Read the data

In [2]:
import pandas as pd
fp = "data/incident_event_log.csv"
df = pd.read_csv(fp)

## What are the main characteristics?
1. What does a sample of the dataset look like?
2. How many incidents are reported in this dataset?

In [3]:
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [4]:
df['number'].nunique()

24918

## List the data types of the various attributes

In [5]:
df.dtypes

number                     object
incident_state             object
active                       bool
reassignment_count          int64
reopen_count                int64
sys_mod_count               int64
made_sla                     bool
caller_id                  object
opened_by                  object
opened_at                  object
sys_created_by             object
sys_created_at             object
sys_updated_by             object
sys_updated_at             object
contact_type               object
location                   object
category                   object
subcategory                object
u_symptom                  object
cmdb_ci                    object
impact                     object
urgency                    object
priority                   object
assignment_group           object
assigned_to                object
knowledge                    bool
u_priority_confirmation      bool
notify                     object
problem_id                 object
rfc           

## Convert the $\texttt{sys_updated_at}$ attribute to be a timestamp

In [6]:
df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at'])

## Machine Learning Task for this Dataset 

The contributors of this dataset have used this data to predict the time to resolution of the ticket. This data has been used for a classification task in this work. A [graph convolutional network for relational data(GCN)](https://arxiv.org/abs/1703.06103) will be the machine learning task for this work. We will be using a __GCN__  to predict the property of a particular node. What property would be useful to predict ? What are the characteristics of this property in the data? The cells below explore these questions. 

### Explore candidate list of tags
Note: For the experiment, we will pick a tag that is fairly evenly distributed in the data. This will avoild the imbalanced classs label problem.

In [7]:
dfcc = df[['made_sla', 'urgency', 'impact', 'reassignment_count']]
for c in dfcc.columns.tolist():
    print(str(dfcc[c].value_counts()))

True     132497
False      9215
Name: made_sla, dtype: int64
2 - Medium    134094
1 - High        4020
3 - Low         3598
Name: urgency, dtype: int64
2 - Medium    134335
3 - Low         3886
1 - High        3491
Name: impact, dtype: int64
0     69876
1     37104
2     15097
3      8274
4      4614
5      2595
6      1447
7       985
8       574
9       365
10      285
11      174
12      108
13       61
14       45
15       21
17       16
20       16
16       13
18       13
22        9
19        8
21        3
27        3
26        2
23        2
24        1
25        1
Name: reassignment_count, dtype: int64


A review of the level counts of the categorical variables in this dataset suggest that $\texttt{made_sla}$ and $\texttt{urgency}$ are both highly imbalanced. The minority levels are almost anomalies. The $\texttt{reassignment_count}$ seems promising. We can derive a new attribute $\texttt{reassigned}$ that captures if the ticket has been reassigned, i.e., has it been assigned to someone after the initial assignment. Such an attribute captures inefficiencies in triaging the ticket and is a useful indicator to track for an organization. A $0$ for this attribute indicates that there was no reassignment and a $1$ indicates that there was a reassignment. This attribute has a nice even spread in the data, i.e., an almost even spread of $0$ and $1$. The cells below create this attribute

## Feature Creation (reassigned):
It looks like tracking ticket reassignment can create a variable that is somewhat evenly distributed in the data. About half the tickets have the correct assignment at first. About half are reassigned to various degrees.

In [8]:
df['reassigned'] = df['reassignment_count'].apply(lambda x: 0 if x == 0 else 1)
df['reassigned'].value_counts()

1    71836
0    69876
Name: reassigned, dtype: int64

In [9]:
dfpp = df.loc[df.groupby(by=['number']).sys_updated_at.idxmax()]
dfpp = dfpp.reset_index()
cols = dfpp.columns.tolist()
cols.remove('index')
cols.remove('number')
dfpp = dfpp[cols]

Now that we have characterized the data and identified the machine learning task to be performed. The next step is to transform the data to a form amenable for machine learning. 